In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
#from midas import m
#from datetime import datetime, date
#import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024"

# SOW202 TFv2 update (different specificity cutoffs 98%, 97%, 95% and 90%)

## Data preocessing

In [2]:
# Readin latest batches (TFv2 updated version by Evan) from Google drive

# Infinity batch
df_sow202_all = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20241114_AstraZeneca_SOW202_Batch1_80Samples_InfinityReport.csv", header = 1) 

print("Total sample size of SOW202 Infinity cumulative batches is " + str(df_sow202_all.shape[0]) + "\n") 
print("Total unique sample size of SOW202 Infinity cumulative batches is " + str(df_sow202_all.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of SOW202 Infinity cumulative batches is " + str(df_sow202_all.Patient_ID.nunique()) + "\n")

df_sow202_all.head()

Total sample size of SOW202 Infinity cumulative batches is 1776

Total unique sample size of SOW202 Infinity cumulative batches is 80

Total unique patient sample size of SOW202 Infinity cumulative batches is 80



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,TNFAIP3,6,138202447.0,9,M788I,G>A,c.2364G>A,NM_001270507.2,52.65,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52800632,rs143002189,NaN,NaN,NaN,NaN,1176.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
1,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,DOT1L,19,2226694.0,27,G1392S,G>A,c.4174G>A,NM_032482.3,51.59,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs375002753,NaN,NaN,NaN,NaN,1131.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
2,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,RET,10,43615594.0,15,S891S,G>A,c.2673G>A,NM_020975.6,51.21,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs201620214,Likely_benign,36302.0,NaN,NaN,1428.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
3,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,ARID2,12,46233253.0,11,T491S,C>G,c.1472C>G,NM_152641.4,50.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV104636884,rs149755754,Likely_benign,137297.0,NaN,NaN,1313.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
4,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,CREBBP,16,3779687.0,31,N1787N,G>A,c.5361C>T,NM_004380.3,49.92,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52129505,rs375462934,Uncertain_significance,100953.0,NaN,NaN,1328.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable


In [3]:
# Remove duplication
df_sow202_dedup = df_sow202_all.drop_duplicates(subset = "GHSampleID")
df_sow202_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,TNFAIP3,6,138202447.0,9,M788I,G>A,c.2364G>A,NM_001270507.2,52.65,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52800632,rs143002189,NaN,NaN,NaN,NaN,1176.0,0.00,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
11,SOW202,34081_0,A1187755,B00817862,34081,0,1,SUCCESS,NaN,SNV,NaN,AXL,19,41727943.0,4,R190C,C>T,c.568C>T,NM_001699.6,51.62,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV99997347,rs770407415,NaN,NaN,NaN,NaN,5775.0,0.37,NaN,NaN,Detected,0.002900,NaN,Not evaluable,Not Detected,73.00,2.5,1.0,10/23/2024,02/23/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
40,SOW202,34791_0,A1187756,B00817848,34791,0,1,SUCCESS,Possible Contamination Warning,SNV,NaN,MLST8,16,2258296.0,7,T220M,C>T,c.659C>T,NM_001199173.3,52.22,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs761521532,NaN,NaN,NaN,NaN,1865.0,5.51,NaN,NaN,Detected,0.001167,NaN,Not evaluable,Not Detected,16.47,3.0,0.0,10/23/2024,04/18/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
61,SOW202,34105_0,A1187757,B00817850,34105,0,1,SUCCESS,NaN,SNV,NaN,LRP6,12,12284874.0,18,N1284S,T>C,c.3851A>G,NM_002336.3,51.06,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs570231124,NaN,NaN,NaN,NaN,4745.0,2.65,NaN,NaN,Detected,0.001108,NaN,Not evaluable,Not Detected,40.18,3.0,0.0,10/23/2024,03/08/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
91,SOW202,34103_0,A1187758,B00817843,34103,0,1,SUCCESS,NaN,SNV,NaN,IRS2,13,110435794.0,1,P869P,G>T,c.2607C>A,NM_003749.3,54.08,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs531803302,Benign,738933.0,NaN,NaN,2207.0,6.97,NaN,NaN,Detected,0.000720,16.07,Low,Not Detected,17.70,3.0,0.0,10/23/2024,03/06/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable


In [4]:
# Pull "runid" from ghdb
df_sow202_1 = df_sow202_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_sow202_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_sow202_1 = df_sow202_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_sow202_1 = df_sow202_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_sow202_1 = df_sow202_1.drop(columns='run_date')

print("Total sample size of SOW202 Infinity cumulative batches after merging runid is " + str(df_sow202_1.shape[0])) 
df_sow202_1.head()


Total sample size of SOW202 Infinity cumulative batches after merging runid is 80


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
26,SOW202,130920290_0,A1187780,B00817816,130920290,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145737441.0,19,F1082F,G>A,c.3246C>T,NM_004260.4,51.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748604051,NaN,NaN,NaN,NaN,4120.0,1.11,NaN,NaN,Detected,0.003186,3.86,Low,Not Detected,98.70,2.5,0.0,10/23/2024,07/05/2023,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A00770_1052_BHTCLWDSXC
65,SOW202,34095_0,A1187819,B00817817,34095,0,1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,SNV,NaN,TFE3,X,48887805.0,10,G531E,C>T,c.1592G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs782285732,NaN,NaN,NaN,NaN,263.0,48.40,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,2.56,3.0,0.0,10/23/2024,02/28/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC
70,SOW202,34800_0,A1187824,B00817818,34800,0,1,SUCCESS,NaN,SNV,NaN,CIC,19,42777840.0,2,P635P,A>G,c.1905A>G,NM_001304815.1,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs926431641,NaN,NaN,NaN,NaN,3204.0,5.59,NaN,NaN,Not Detected,0.000000,10.40,Low,Not Detected,24.47,3.0,0.0,10/23/2024,05/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC
71,SOW202,34070_0,A1187825,B00817819,34070,0,1,SUCCESS,NaN,SNV,NaN,CDKN1C,11,2906208.0,1,A171V,G>A,c.512C>T,NM_000076.2,80.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,47.17,NaN,NaN,Not Detected,0.000000,6.62,Low,Not Detected,61.32,3.0,0.0,10/23/2024,02/29/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC
68,SOW202,34118_0,A1187822,B00817820,34118,0,1,SUCCESS,NaN,SNV,NaN,ARID1B,6,157222571.0,3,Y600C,A>G,c.1799A>G,NM_017519.3,50.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4508.0,0.24,NaN,NaN,Detected,0.000202,NaN,Not evaluable,Not Detected,91.10,2.5,0.0,10/23/2024,04/05/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC


In [5]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        if folders:
            for folder in folders:
                primary_fc_dir = folder
                # Check if there is a 'sirius' folder under the primary fc_dir. This is for Sirius 1.5 folder structure change.
                if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                    primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
                df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_sow202_2, df_no_found = add_fc_dir_column(df_sow202_1)

print("Total sample size of SOW202 Infinity cumulative batches with fc_dir is " + str(df_sow202_2.shape[0]) + "\n") 
print("Total sample size of SOW202 Infinity cumulative batches with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_sow202_2.head()



Total sample size of SOW202 Infinity cumulative batches with fc_dir is 80

Total sample size of SOW202 Infinity cumulative batches with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
26,SOW202,130920290_0,A1187780,B00817816,130920290,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145737441.0,19,F1082F,G>A,c.3246C>T,NM_004260.4,51.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748604051,NaN,NaN,NaN,NaN,4120.0,1.11,NaN,NaN,Detected,0.003186,3.86,Low,Not Detected,98.70,2.5,0.0,10/23/2024,07/05/2023,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A00770_1052_BHTCLWDSXC,/ghsfa/ivd/flowcentral/241101_A00770_1052_BHTCLWDSXC.79230695-5143-4a73-b52e-a27b1400da85.20241103143625/sirius
65,SOW202,34095_0,A1187819,B00817817,34095,0,1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,SNV,NaN,TFE3,X,48887805.0,10,G531E,C>T,c.1592G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs782285732,NaN,NaN,NaN,NaN,263.0,48.40,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,2.56,3.0,0.0,10/23/2024,02/28/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius
70,SOW202,34800_0,A1187824,B00817818,34800,0,1,SUCCESS,NaN,SNV,NaN,CIC,19,42777840.0,2,P635P,A>G,c.1905A>G,NM_001304815.1,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs926431641,NaN,NaN,NaN,NaN,3204.0,5.59,NaN,NaN,Not Detected,0.000000,10.40,Low,Not Detected,24.47,3.0,0.0,10/23/2024,05/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius
71,SOW202,34070_0,A1187825,B00817819,34070,0,1,SUCCESS,NaN,SNV,NaN,CDKN1C,11,2906208.0,1,A171V,G>A,c.512C>T,NM_000076.2,80.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,47.17,NaN,NaN,Not Detected,0.000000,6.62,Low,Not Detected,61.32,3.0,0.0,10/23/2024,02/29/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius
68,SOW202,34118_0,A1187822,B00817820,34118,0,1,SUCCESS,NaN,SNV,NaN,ARID1B,6,157222571.0,3,Y600C,A>G,c.1799A>G,NM_017519.3,50.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4508.0,0.24,NaN,NaN,Detected,0.000202,NaN,Not evaluable,Not Detected,91.10,2.5,0.0,10/23/2024,04/05/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius


## Generate Lung S3 MB caller outputs for this SOW (NSCLC)
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update

In [ ]:
# Instead of using epicaller.methylation.methyl_caller funciton, bip-epigenomics module (branch 2.5 with Sirius 1.6 MB caller region/control/module files) is prefered.

def run_bip_epigenomics_module_wrapper_slurm(sample_meta_df, workdir):
    module_dir = '/ghsfa/projects/omni_v2/users/chlee/bip4/bip-epigenomics/' # branch 2.5
    # parameter_set_yaml = "/ghsfa/bik8s/test/bip-portal/outputs/Sirius_LDT_Soft_240905_020053/products/sirius/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # branch Sirius 1.6
    parameter_set_yaml = "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_crossSOW_09042024/bip-products/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # branch Sirius 1.6


    jobs_dir = f"{workdir}/MB_module_SOW202_jobs"
    output_dir = f"{workdir}/MB_module_SOW202_output"

    # Check and create directories if not exists
    if not os.path.exists(jobs_dir):
        os.makedirs(jobs_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]

        # # Check if there is a 'sirius' folder under the fc_dir. This is for Sirius 1.5 update
        # if os.path.exists(os.path.join(fc_dir, "sirius")):
        #     fc_dir = os.path.join(fc_dir, "sirius")

        mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
        bip_config_json_file = f"{fc_dir}/bip_config.json"
        
        if not os.path.exists(mbd_hyper_molecules):
            continue

        # Make a folder for each run_sample_id
        output_sample_dir = f"{output_dir}/{run_sample_id}"

        if not os.path.exists(output_sample_dir):
            os.makedirs(output_sample_dir)

        # Copy the bip_config.json for each sample from BIP output. This is essential for module running
        # (samples in the same batch shared the same bip_config.json)
        os.system(f"cp {bip_config_json_file} {output_sample_dir}")

        job_file = f"{jobs_dir}/MB_call_{run_sample_id}.job"
        shebang_content = (f"#!/bin/bash\n"
                           f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
                           f"#SBATCH --nodes=1\n"
                           f"#SBATCH --ntasks=1\n"
                           f"#SBATCH --cpus-per-task=1\n"
                           f"#SBATCH --mem=60gb\n"
                           f"#SBATCH --partition=dev\n"
                           f"#SBATCH --time=01:00:00\n"
                           f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n\n")
        
        with open(job_file, "w") as job_output:
            job_output.write(shebang_content)
            job_output.write(f"{module_dir}/module_entry msre_caller " + 
                             f"--in-hyper-molecules {mbd_hyper_molecules} " + 
                             f"--sample-id {run_sample_id} " + 
                             f"--output-dir {output_sample_dir}/ " +
                             f"--parameter-set {parameter_set_yaml} " + 
                             f"--threads 16 " + 
                             f"--memory-log {output_sample_dir}/{run_sample_id}-epigenomics_msre_caller.memory.log " +
                             f"--log-file {output_sample_dir}/{run_sample_id}--epigenomics_msre_caller.log")
            # print(job_output)    
        os.system(f"sbatch {job_file}")


In [ ]:
#run the function to generate all the MB S3 caller ouputs for this sow  
run_bip_epigenomics_module_wrapper_slurm(df_sow202_2, 
                      f"{work_dir}")

Submitted batch job 93072519
Submitted batch job 93072520
Submitted batch job 93072521
Submitted batch job 93072522
Submitted batch job 93072523
Submitted batch job 93072524
Submitted batch job 93072525
Submitted batch job 93072526
Submitted batch job 93072527
Submitted batch job 93072528
Submitted batch job 93072529
Submitted batch job 93072530
Submitted batch job 93072531
Submitted batch job 93072532
Submitted batch job 93072533
Submitted batch job 93072534
Submitted batch job 93072535
Submitted batch job 93072536
Submitted batch job 93072537
Submitted batch job 93072538
Submitted batch job 93072539
Submitted batch job 93072540
Submitted batch job 93072541
Submitted batch job 93072542
Submitted batch job 93072543
Submitted batch job 93072544
Submitted batch job 93072545
Submitted batch job 93072546
Submitted batch job 93072547
Submitted batch job 93072548
Submitted batch job 93072549
Submitted batch job 93072550
Submitted batch job 93072551
Submitted batch job 93072552
Submitted batc

In [6]:
# Retrieve MB S3 lung caller score

df_sow202_3 = df_sow202_2.copy()

df_sow202_3["MB_lung_score"] = np.nan

mb_caller_result_folder = f"{work_dir}/MB_module_SOW202_output"
problematic_rows = []  # List to store indices of problematic rows

for index, crc_row in df_sow202_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]
    msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_lung_call.hdr.tsv"

    try:
        msre_df = pd.read_table(msre_mr_file)

        if not msre_df.empty:
            df_sow202_3.at[index, "MB_lung_score"] = msre_df.loc[0, "methyl_score"]

        else:
            # Add the index to the list if msre_df is empty
            problematic_rows.append(index) 
            
    except Exception as e:
        # Add the index to the list if there is any other exception
        problematic_rows.append(index)  
        print(f"Error processing file {msre_mr_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_sow202_3.loc[problematic_rows]

print("Total sample size of SOW202 Infinity cumulative batches with NO MB output is " + str(df_problematic.shape[0]) + "\n") 

# Returning the DataFrame with problematic rows
df_sow202_3.head()


Total sample size of SOW202 Infinity cumulative batches with NO MB output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_lung_score
26,SOW202,130920290_0,A1187780,B00817816,130920290,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145737441.0,19,F1082F,G>A,c.3246C>T,NM_004260.4,51.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748604051,NaN,NaN,NaN,NaN,4120.0,1.11,NaN,NaN,Detected,0.003186,3.86,Low,Not Detected,98.70,2.5,0.0,10/23/2024,07/05/2023,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A00770_1052_BHTCLWDSXC,/ghsfa/ivd/flowcentral/241101_A00770_1052_BHTCLWDSXC.79230695-5143-4a73-b52e-a27b1400da85.20241103143625/sirius,-21.126880
65,SOW202,34095_0,A1187819,B00817817,34095,0,1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,SNV,NaN,TFE3,X,48887805.0,10,G531E,C>T,c.1592G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs782285732,NaN,NaN,NaN,NaN,263.0,48.40,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,2.56,3.0,0.0,10/23/2024,02/28/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-22.610623
70,SOW202,34800_0,A1187824,B00817818,34800,0,1,SUCCESS,NaN,SNV,NaN,CIC,19,42777840.0,2,P635P,A>G,c.1905A>G,NM_001304815.1,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs926431641,NaN,NaN,NaN,NaN,3204.0,5.59,NaN,NaN,Not Detected,0.000000,10.40,Low,Not Detected,24.47,3.0,0.0,10/23/2024,05/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-21.885983
71,SOW202,34070_0,A1187825,B00817819,34070,0,1,SUCCESS,NaN,SNV,NaN,CDKN1C,11,2906208.0,1,A171V,G>A,c.512C>T,NM_000076.2,80.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,47.17,NaN,NaN,Not Detected,0.000000,6.62,Low,Not Detected,61.32,3.0,0.0,10/23/2024,02/29/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.411342
68,SOW202,34118_0,A1187822,B00817820,34118,0,1,SUCCESS,NaN,SNV,NaN,ARID1B,6,157222571.0,3,Y600C,A>G,c.1799A>G,NM_017519.3,50.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4508.0,0.24,NaN,NaN,Detected,0.000202,NaN,Not evaluable,Not Detected,91.10,2.5,0.0,10/23/2024,04/05/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.587837


## Generate CpG lung caller outputs for this SOW (NSCLC)

In [ ]:
import epicaller
print(epicaller.__version__)

2.0.3


In [10]:
#The input files (region_file, lr_model_file, control_region_file and mr_model_file were got from Shile (Elmira) check slack conversation
import epicaller
import epicaller.methylation.methyl_caller

def run_CpG_lung_caller_wrapper_slurm(sample_meta_df, mr_model_file, workdir):
    region_file='/screening/data/methylation/sandbox_pipeline/assets/s3_cpg_clusters_breast_lung_v0.0.2/cpg_clusters.tsv'
    control_region_file='/screening/data/methylation/sandbox_pipeline/assets/s3_full_regions_v0.0.2/v5_model_control_regions.tsv'
    region_group_file='/screening/data/methylation/sandbox_pipeline/assets/s3_cpg_clusters_breast_lung_v0.0.2/cpg_cluster_groups.tsv'

    jobs_dir = f"{workdir}/CpG_Lung_log"
    output_dir = f"{workdir}/CpG_Lung_output"

    # Check and create directories if not exists
    if not os.path.exists(jobs_dir):
        os.makedirs(jobs_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        #This is to deal the the cluster migration issue
        fc_dir = row["fc_dir"].replace("/ghess", "/ghsfa/projects") 
        flowcell_id = row["runid"]
        mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
        # mbd_residual_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_residual_molecules.tsv"
        
        # Check if the molecule file exists
        if not os.path.exists(mbd_hyper_molecules):
            continue

        job_file = f"{jobs_dir}/CgG_lung_call_{run_sample_id}.job"
        shebang_content = (f"#!/bin/bash\n"
                           f"#SBATCH --job-name=CpG_lung_call_{run_sample_id}\n"
                           f"#SBATCH --nodes=1\n"
                           f"#SBATCH --ntasks=1\n"
                           f"#SBATCH --cpus-per-task=1\n"
                           f"#SBATCH --mem=30gb\n"
                           f"#SBATCH --partition=dev\n"
                           f"#SBATCH --time=00:10:00\n"
                           f"#SBATCH --output={jobs_dir}/CpG_lung_call_{run_sample_id}.log\n\n")
        
        with open(job_file, "w") as job_output:
            job_output.write(shebang_content)
            job_output.write("python3 -m epicaller.methylation.methyl_caller " + 
                             f"--hyper-molecule-file {mbd_hyper_molecules} " + 
                             f"--region-file {region_file} " +
                             f"--control-region-file {control_region_file} " + 
                             f"--model-file {mr_model_file} " +
                             "--model-name lung_cpg --min-frag-len 120 --max-frag-len 240 --min-cg-count 12 --max-cg-count 30 --cutsite-count-cutoff 1 --min-mapq 60 --intersect-region-frac 1.0 " + 
                             f"--sample-id {run_sample_id} --flowcell-id {flowcell_id} --region-group-file {region_group_file} --outdir {output_dir}/")
            # print(job_output)    
        os.system(f"sbatch {job_file}")


In [11]:
#run the function to generate all the GpG Lung caller ouputs for NAD02_IST01_Infinity_01 batch 
run_CpG_lung_caller_wrapper_slurm(df_sow202_2, 
                      "/screening/analysis/2024_05_05_MRD_AV_ReAnalysis/metadata/Lung_MB_5_v1.20_mendocino_tvf_logit_maf_cgp_lung_top500_and_standard_v6.model_file.98_spec_set_on_s3_data.tsv", 
                      f"{work_dir}")

Submitted batch job 93107655
Submitted batch job 93107656
Submitted batch job 93107657
Submitted batch job 93107658
Submitted batch job 93107659
Submitted batch job 93107660
Submitted batch job 93107661
Submitted batch job 93107662
Submitted batch job 93107663
Submitted batch job 93107664
Submitted batch job 93107665
Submitted batch job 93107666
Submitted batch job 93107667
Submitted batch job 93107668
Submitted batch job 93107669
Submitted batch job 93107670
Submitted batch job 93107671
Submitted batch job 93107672
Submitted batch job 93107673
Submitted batch job 93107674
Submitted batch job 93107675
Submitted batch job 93107676
Submitted batch job 93107677
Submitted batch job 93107678
Submitted batch job 93107679
Submitted batch job 93107680
Submitted batch job 93107681
Submitted batch job 93107682
Submitted batch job 93107683
Submitted batch job 93107684
Submitted batch job 93107685
Submitted batch job 93107686
Submitted batch job 93107687
Submitted batch job 93107688
Submitted batc

In [19]:
# Retrieve MB S3 lung caller score

df_sow202_3 = df_sow202_2.copy()

df_sow202_3["CpG_lung_score"] = np.nan

mb_caller_result_folder = f"{work_dir}/CpG_Lung_output"
problematic_rows = []  # List to store indices of problematic rows

for index, crc_row in df_sow202_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]
    msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}.msre_lung_cpg_call.hdr.tsv"

    try:
        msre_df = pd.read_table(msre_mr_file)

        if not msre_df.empty:
            df_sow202_3.at[index, "CpG_lung_score"] = msre_df.loc[0, "methyl_score"]

        else:
            # Add the index to the list if msre_df is empty
            problematic_rows.append(index) 
            
    except Exception as e:
        # Add the index to the list if there is any other exception
        problematic_rows.append(index)  
        print(f"Error processing file {msre_mr_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_sow202_3.loc[problematic_rows]

print("Total sample size of SOW202 Infinity cumulative batches with NO CpG caller output is " + str(df_problematic.shape[0]) + "\n") 

# Returning the DataFrame with problematic rows
df_sow202_3.head()


Total sample size of SOW202 Infinity cumulative batches with NO CpG caller output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,CpG_lung_score
26,SOW202,130920290_0,A1187780,B00817816,130920290,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145737441.0,19,F1082F,G>A,c.3246C>T,NM_004260.4,51.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748604051,NaN,NaN,NaN,NaN,4120.0,1.11,NaN,NaN,Detected,0.003186,3.86,Low,Not Detected,98.70,2.5,0.0,10/23/2024,07/05/2023,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A00770_1052_BHTCLWDSXC,/ghsfa/ivd/flowcentral/241101_A00770_1052_BHTCLWDSXC.79230695-5143-4a73-b52e-a27b1400da85.20241103143625/sirius,-20.639350
65,SOW202,34095_0,A1187819,B00817817,34095,0,1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,SNV,NaN,TFE3,X,48887805.0,10,G531E,C>T,c.1592G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs782285732,NaN,NaN,NaN,NaN,263.0,48.40,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,2.56,3.0,0.0,10/23/2024,02/28/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-21.641157
70,SOW202,34800_0,A1187824,B00817818,34800,0,1,SUCCESS,NaN,SNV,NaN,CIC,19,42777840.0,2,P635P,A>G,c.1905A>G,NM_001304815.1,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs926431641,NaN,NaN,NaN,NaN,3204.0,5.59,NaN,NaN,Not Detected,0.000000,10.40,Low,Not Detected,24.47,3.0,0.0,10/23/2024,05/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-22.053279
71,SOW202,34070_0,A1187825,B00817819,34070,0,1,SUCCESS,NaN,SNV,NaN,CDKN1C,11,2906208.0,1,A171V,G>A,c.512C>T,NM_000076.2,80.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,47.17,NaN,NaN,Not Detected,0.000000,6.62,Low,Not Detected,61.32,3.0,0.0,10/23/2024,02/29/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.212699
68,SOW202,34118_0,A1187822,B00817820,34118,0,1,SUCCESS,NaN,SNV,NaN,ARID1B,6,157222571.0,3,Y600C,A>G,c.1799A>G,NM_017519.3,50.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4508.0,0.24,NaN,NaN,Detected,0.000202,NaN,Not evaluable,Not Detected,91.10,2.5,0.0,10/23/2024,04/05/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.461339


## Generate the epiTFv2 gamma score results

In [21]:
# Temporary solution for TFv2gamma score generation
df_sow202_3 = df_sow202_3[df_sow202_3['Sample_status'] != "FAIL"]
df_sow202_3.shape

# Save the data
df_sow202_3.to_csv(f"{work_dir}/df_sow202_3.csv", index = False)

In [18]:
# Step 1: save the input csv file
df_sow202_3 = pd.read_csv(f"{work_dir}/df_sow202_3.csv", header = 0)
df_sow202_3.head()

# Step 2 generate the slurm script
TFv2gamma_output_dir = f"{work_dir}/TFv2gamma_SOW202_output"

# Check and create directories if not exists
if not os.path.exists(TFv2gamma_output_dir):
    os.makedirs(TFv2gamma_output_dir)

In [ ]:
# Step 2 generate the slurm script

# Define the contents of the slurm script
# Parts to updates
    # sample_list; out_path; slurm_script_path

slurm_script_content = f"""#! /bin/sh
#SBATCH --get-user-env
#SBATCH --job-name TFv2gamma_pancanTF
#SBATCH --partition dev
#SBATCH --mem-per-cpu 40G
#SBATCH --cpus-per-task 1
#SBATCH --ntasks-per-node 1
#SBATCH -o {TFv2gamma_output_dir}/epiTFv2gamma_%A.log
#SBATCH --array 1-1000%300

module load python/3.9.6;
export INSTALL_DIR=/home/mcai/workspace/tfv2_gamma/bip-pycommon;
source $INSTALL_DIR/.py3_venv/bin/activate;
export PYTHONPATH=$INSTALL_DIR:$PYTHONPATH;

sample_list={work_dir}/df_sow202_3.csv
out_path="{TFv2gamma_output_dir}"
mkdir -p $out_path

# Use Python to extract the necessary fields robustly
# Note that the parsing method is changed due to weired parsing issue of "fc_dir" of this sow
read sample_id fc_dir <<< $(python -c "import pandas as pd; import os
df = pd.read_csv('$sample_list')
# Adjusting SLURM_ARRAY_TASK_ID from 1-based to 0-based index
index = int(os.getenv('SLURM_ARRAY_TASK_ID', '1')) - 1
sample_id = df.iloc[index]['GHSampleID']
fc_dir = df.iloc[index]['fc_dir']
print(sample_id, fc_dir)")

# Prepend base path if fc_dir is relative
base_path="/ghsfa/ivd/flowcentral"
if [[ "$fc_dir" != /* ]]; then
    fc_dir="${{base_path}}/${{fc_dir}}"
fi

# Construct paths
sample_path="${{fc_dir}}/${{sample_id}}"
mbd_path="${{sample_path}}/${{sample_id}}.mbd_hyper_molecules.tsv"
methyl_qc_file="${{sample_path}}/${{sample_id}}.methyl_qc.tsv"
subpanel_qc_path="${{mbd_path/mbd_hyper_molecules/subpanel_qc_stats.hdr}}"

# Run the code
python -m epi_tumor_fraction.run_epi_tf $mbd_path $methyl_qc_file $out_path $sample_id /ghdevhome/home/mcai/workspace/tfv2_gamma/bip-products/models/Sirius/epi_tf_models/params.sirius.yaml --subpanel_qc_file $subpanel_qc_path
"""

# Save the slurm script to file
slurm_script_path = f"{work_dir}/run_TFv2gamma_sow202.sh"
with open(slurm_script_path, "w") as file:
    file.write(slurm_script_content)

In [ ]:
# Step 3 run the script
os.system(f"sbatch {work_dir}/run_TFv2gamma_sow202.sh")

Submitted batch job 93073390


0

In [20]:
# Retrive the TFv2gamma value

# Remove the "FAIL" samples
df_sow202_4 = df_sow202_3[df_sow202_3["Sample_status"] != "FAIL"]

# Initialize a list to store rows where no TFv2gamma output was found
rows_no_tfv2 = []

df_sow202_4['pred_frac_force'] = None 

TFv2gamma_output_dir = f"{work_dir}/TFv2gamma_SOW202_output"

for index, row in df_sow202_4.iterrows():
    sample_id = row["GHSampleID"]
    file_path = f"{TFv2gamma_output_dir}/{sample_id}.epi_tf_model_predictions.tsv"

    file_path_tfv2 = glob.glob(file_path)
    if not file_path_tfv2:
        rows_no_tfv2.append(row)
    
    else:
        try:
            df_temp = pd.read_table(file_path)
            value_gamma = df_temp[df_temp['metric'] == 'epi_tf']['value'].values[0]
            df_sow202_4.at[index, 'pred_frac_force'] = value_gamma
            
        except Exception as e:
            print(f"An error occurred with {sample_id}: {str(e)}")

# Create a DataFrame with rows where folders were not found in any sources
df_no_tfv2 = pd.DataFrame(rows_no_tfv2)
print("Total sample size of SOW202 Infinity cumulative batches with NO TFv2gamma output is " + str(df_no_tfv2.shape[0]) + "\n") 


df_sow202_5 = df_sow202_4.copy()
df_sow202_5.head()

Total sample size of SOW202 Infinity cumulative batches with NO TFv2gamma output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,CpG_lung_score,pred_frac_force
26,SOW202,130920290_0,A1187780,B00817816,130920290,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145737441.0,19,F1082F,G>A,c.3246C>T,NM_004260.4,51.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748604051,NaN,NaN,NaN,NaN,4120.0,1.11,NaN,NaN,Detected,0.003186,3.86,Low,Not Detected,98.70,2.5,0.0,10/23/2024,07/05/2023,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A00770_1052_BHTCLWDSXC,/ghsfa/ivd/flowcentral/241101_A00770_1052_BHTCLWDSXC.79230695-5143-4a73-b52e-a27b1400da85.20241103143625/sirius,-20.639350,0.010985
65,SOW202,34095_0,A1187819,B00817817,34095,0,1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,SNV,NaN,TFE3,X,48887805.0,10,G531E,C>T,c.1592G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs782285732,NaN,NaN,NaN,NaN,263.0,48.40,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,2.56,3.0,0.0,10/23/2024,02/28/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-21.641157,0.000143
70,SOW202,34800_0,A1187824,B00817818,34800,0,1,SUCCESS,NaN,SNV,NaN,CIC,19,42777840.0,2,P635P,A>G,c.1905A>G,NM_001304815.1,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs926431641,NaN,NaN,NaN,NaN,3204.0,5.59,NaN,NaN,Not Detected,0.000000,10.40,Low,Not Detected,24.47,3.0,0.0,10/23/2024,05/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-22.053279,0.00012
71,SOW202,34070_0,A1187825,B00817819,34070,0,1,SUCCESS,NaN,SNV,NaN,CDKN1C,11,2906208.0,1,A171V,G>A,c.512C>T,NM_000076.2,80.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,47.17,NaN,NaN,Not Detected,0.000000,6.62,Low,Not Detected,61.32,3.0,0.0,10/23/2024,02/29/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.212699,0.041027
68,SOW202,34118_0,A1187822,B00817820,34118,0,1,SUCCESS,NaN,SNV,NaN,ARID1B,6,157222571.0,3,Y600C,A>G,c.1799A>G,NM_017519.3,50.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4508.0,0.24,NaN,NaN,Detected,0.000202,NaN,Not evaluable,Not Detected,91.10,2.5,0.0,10/23/2024,04/05/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.461339,0.000451


### For checking troubleshooting purposes on TFv2gamma (no need to run)


In [ ]:
# This is for checking troubleshooting purposes

# Sample that has no TFv2gamma output "B00481373"
df_sow164_4 = df_sow187_3[df_sow187_3["GHSampleID"] == "B00625530"]
# df_sow157_4 = df_sow157_3[df_sow157_3["GHSampleID"] == "B00558789"]

# Save the data for test sh run
df_sow164_4.to_csv(f"{work_dir}/df_sow164_4.csv", index = False)
df_sow164_4

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,Tumor_stage,ctDNA_detected,cfDNA_ng_yield,Unnamed: 0,runid,fc_dir,Tumor_methylation_status_mapped,MB_score
2,D926UC00001,6802077988,B00625530,B00625530,E4306001,SCREEN,1.0,SUCCESS,NaN,SNV,NaN,SOX2,3,181430551.0,1.0,G135S,G>A,c.403G>A,NM_003106.4,50.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2767.0,0.14,NaN,NaN,NaN,NaN,NaN,Not evaluable,Not Detected,75.92,1.0,NaN,01/10/2024,NaN,07/17/2024,NaN,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,240619_A01744_0224_AHG3Y7DSXC,/ghsfa/ivd/flowcentral/240619_A01744_0224_AHG3Y7DSXC.5cae61cd-3024-4c67-8bcc-d1e799ee5671.20240626013927,NaN,-22.813004


In [ ]:
print(df_sow164_4['fc_dir'])


2    /ghsfa/ivd/flowcentral/240619_A01744_0224_AHG3Y7DSXC.5cae61cd-3024-4c67-8bcc-d1e799ee5671.20240626013927
Name: fc_dir, dtype: object


In [ ]:
# STEP 3: run the test script with debugging prints
# os.system(f"sbatch {work_dir}/run_TFv2gamma_pancanTF_test187.sh")
os.system(f"sbatch {work_dir}/run_TFv2gamma_pancanTF_test187_new_1.sh")

Submitted batch job 87999489


0

## Generate the updated report of the SOW based on "TFv2" rule 

In [15]:
def update_tumor_methylation_status_MBcutoff_withTFv2filter(df, MB_score_column, MB18_cutoff, MB_cutoff):
    """
    Update the 'Tumor_methylation_status' and 'Tumor_methylation_score' columns in the DataFrame based on provided cutoff values ('TFv2' rule).
    
    Parameters:
        df (DataFrame): The DataFrame to update. Must with two columns "MB_score_column" and TFv2gamma ("pred_frac_force")
        MB_score_column (str): The column name for MB caller scores.
        MB18_cutoff (float): The cutoff value for the MB18 (must match MB caller type!).
        MB_cutoff (float): The cutoff value for the MB90/MB95/MB97 (must match MB caller type!).

    Notes:
        cutoff refenence: https://docs.google.com/presentation/d/1U6TYnddThQHxo9-Ahxpty2ie2_SpnIEcICbTobH4s4g/edit#slide=id.g2f5b7ef9d14_0_50
                          https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update
        Elmira: Lung model 97%： -20.417943 Lung model 95%： -20.665406 
    """

    # case1: Addressing flipped calls where "Tumor_methylation_status" is "Not Detected" but TFv2 rule is triggered.
            # Then flip it to "Detected" and replace value "Tumor_methylation_score" with "pred_frac_force"
    conditions_case1 = (
        (df['Tumor_methylation_status'] == 'Not Detected') &
        ((df[MB_score_column] >= MB_cutoff) |
         ((df[MB_score_column] >= MB18_cutoff) & (df[MB_score_column] < MB_cutoff) & (df['pred_frac_force'] > 0.0005)))
    )

    # case2: Addressing flipped calls where "Tumor_methylation_status" is "Detected" but TFv2 rule is NOT triggered. 
            # Then flip it to "Not Detected" and replace value "Tumor_methylation_score" with 0
    conditions_case2 = (
        (df['Tumor_methylation_status'] == 'Detected') &
        ~((df[MB_score_column] >= MB_cutoff) |
          ((df[MB_score_column] >= MB18_cutoff) & (df[MB_score_column] < MB_cutoff) & (df['pred_frac_force'] > 0.0005)))
    )

    # case3: "Tumor_methylation_status" is nan for FAIL samples. No updates

    # Apply updates based on conditions
    df.loc[conditions_case1, 'Tumor_methylation_status'] = 'Detected'
    df.loc[conditions_case1, 'Tumor_methylation_score'] = df.loc[conditions_case1, 'pred_frac_force']

    df.loc[conditions_case2, 'Tumor_methylation_status'] = 'Not Detected'
    df.loc[conditions_case2, 'Tumor_methylation_score'] = 0

    # Replace the udpated "Tumor_methylation_score" with "pred_frac_force" when "Tumor_methylation_status" is "Detected"
    df['Tumor_methylation_score'] = np.where(df['Tumor_methylation_status'] == "Detected", df['pred_frac_force'], df['Tumor_methylation_score'])

    return df

In [9]:
# Run MB Lung caller cutoffs for the SOW: MB18: -22.421763; MB90: -20.941267; MB95: -20.665406; MB97: -20.417943
MB18_cutoff = -22.421763
MB90_cutoff = -20.941267
MB95_cutoff = -20.665406
MB97_cutoff = -20.417943
MB98_cutoff = -20.343954721795292

# df_sow202_updated_90_withTFv2filter = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_sow202_5, 'MB_lung_score', MB18_cutoff, MB90_cutoff)
# df_sow202_updated_95_withTFv2filter = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_sow202_5, 'MB_lung_score', MB18_cutoff, MB95_cutoff)
df_sow202_updated_97_withTFv2filter = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_sow202_5, 'MB_lung_score', MB18_cutoff, MB97_cutoff)

# df_sow202_updated = df_sow202_updated_90_withTFv2filter.copy()
# df_sow202_updated = df_sow202_updated_95_withTFv2filter.copy()
df_sow202_updated = df_sow202_updated_97_withTFv2filter.copy()

df_sow202_updated.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_lung_score,pred_frac_force
26,SOW202,130920290_0,A1187780,B00817816,130920290,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145737441.0,19,F1082F,G>A,c.3246C>T,NM_004260.4,51.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748604051,NaN,NaN,NaN,NaN,4120.0,1.11,NaN,NaN,Detected,0.010985,3.86,Low,Not Detected,98.70,2.5,0.0,10/23/2024,07/05/2023,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A00770_1052_BHTCLWDSXC,/ghsfa/ivd/flowcentral/241101_A00770_1052_BHTCLWDSXC.79230695-5143-4a73-b52e-a27b1400da85.20241103143625/sirius,-21.126880,0.010985
65,SOW202,34095_0,A1187819,B00817817,34095,0,1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,SNV,NaN,TFE3,X,48887805.0,10,G531E,C>T,c.1592G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs782285732,NaN,NaN,NaN,NaN,263.0,48.40,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,2.56,3.0,0.0,10/23/2024,02/28/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-22.610623,0.000143
70,SOW202,34800_0,A1187824,B00817818,34800,0,1,SUCCESS,NaN,SNV,NaN,CIC,19,42777840.0,2,P635P,A>G,c.1905A>G,NM_001304815.1,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs926431641,NaN,NaN,NaN,NaN,3204.0,5.59,NaN,NaN,Not Detected,0.0,10.40,Low,Not Detected,24.47,3.0,0.0,10/23/2024,05/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-21.885983,0.00012
71,SOW202,34070_0,A1187825,B00817819,34070,0,1,SUCCESS,NaN,SNV,NaN,CDKN1C,11,2906208.0,1,A171V,G>A,c.512C>T,NM_000076.2,80.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,47.17,NaN,NaN,Detected,0.041027,6.62,Low,Not Detected,61.32,3.0,0.0,10/23/2024,02/29/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.411342,0.041027
68,SOW202,34118_0,A1187822,B00817820,34118,0,1,SUCCESS,NaN,SNV,NaN,ARID1B,6,157222571.0,3,Y600C,A>G,c.1799A>G,NM_017519.3,50.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4508.0,0.24,NaN,NaN,Not Detected,0,NaN,Not evaluable,Not Detected,91.10,2.5,0.0,10/23/2024,04/05/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.587837,0.000451


In [17]:
def update_tumor_methylation_status_MB(df, MB_score_column, MB_cutoff):
    """
    Update the 'Tumor_methylation_status' and 'Tumor_methylation_score' columns in the DataFrame based on provided cutoff values ('TFv2' rule).
    
    Parameters:
        df (DataFrame): The DataFrame to update. Must with two columns "MB_score_column" and TFv2gamma ("pred_frac_force")
        MB_score_column (str): The column name for MB caller scores.
        MB_cutoff (float): The cutoff value for the MB90/MB95/MB97 (must match MB caller type!).

    Notes:
        cutoff refenence: https://docs.google.com/presentation/d/1U6TYnddThQHxo9-Ahxpty2ie2_SpnIEcICbTobH4s4g/edit#slide=id.g2f5b7ef9d14_0_50
                          https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update
        Elmira: Lung model 97%： -20.417943 Lung model 95%： -20.665406 
    """

    # case1: Addressing flipped calls where "Tumor_methylation_status" is "Not Detected" but TFv2 rule is triggered.
            # Then flip it to "Detected" and replace value "Tumor_methylation_score" with "pred_frac_force"
    conditions_case1 = (
        (df['Tumor_methylation_status'] == 'Not Detected') & (df[MB_score_column] >= MB_cutoff)
    )

    # case2: Addressing flipped calls where "Tumor_methylation_status" is "Detected" but TFv2 rule is NOT triggered. 
            # Then flip it to "Not Detected" and replace value "Tumor_methylation_score" with 0
    conditions_case2 = (
        (df['Tumor_methylation_status'] == 'Detected') & ~(df[MB_score_column] >= MB_cutoff)
    )

    # case3: "Tumor_methylation_status" is nan for FAIL samples. No updates

    # Apply updates based on conditions
    df.loc[conditions_case1, 'Tumor_methylation_status'] = 'Detected'
    df.loc[conditions_case1, 'Tumor_methylation_score'] = df.loc[conditions_case1, 'pred_frac_force']

    df.loc[conditions_case2, 'Tumor_methylation_status'] = 'Not Detected'
    df.loc[conditions_case2, 'Tumor_methylation_score'] = 0

    # Replace the udpated "Tumor_methylation_score" with "pred_frac_force" when "Tumor_methylation_status" is "Detected"
    df['Tumor_methylation_score'] = np.where(df['Tumor_methylation_status'] == "Detected", df['pred_frac_force'], df['Tumor_methylation_score'])

    return df

In [ ]:
# Run Lung caller cutoffs for the SOW
# df_sow202_updated_95 = update_tumor_methylation_status_MB(df_sow202_5, 'MB_lung_score', MB95_cutoff)
# df_sow202_updated_97 = update_tumor_methylation_status_MB(df_sow202_5, 'MB_lung_score', MB97_cutoff)
df_sow202_updated_98 = update_tumor_methylation_status_MB(df_sow202_5, 'MB_lung_score', MB98_cutoff)

# df_sow202_updated = df_sow202_updated_95.copy()
# df_sow202_updated = df_sow202_updated_97.copy()
df_sow202_updated = df_sow202_updated_98.copy()


df_sow202_updated.head()

In [28]:
# Run CpG Lung caller cutoffs for the SOW
CpG90_cutoff = -21.385639423732425
CpG95_cutoff = -21.195718
CpG97_cutoff = -21.059829
CpG98_cutoff = -20.983137163819325

# df_sow202_updated_CpG_90 = update_tumor_methylation_status_MB(df_sow202_5, 'CpG_lung_score', CpG90_cutoff)
# df_sow202_updated_CpG_95 = update_tumor_methylation_status_MB(df_sow202_5, 'CpG_lung_score', CpG95_cutoff)
df_sow202_updated_CpG_97 = update_tumor_methylation_status_MB(df_sow202_5, 'CpG_lung_score', CpG97_cutoff)
# df_sow202_updated_CpG_98 = update_tumor_methylation_status_MB(df_sow202_5, 'CpG_lung_score', CpG98_cutoff)

# df_sow202_updated = df_sow202_updated_CpG_90.copy()
# df_sow202_updated = df_sow202_updated_CpG_95.copy()
df_sow202_updated = df_sow202_updated_CpG_97.copy()
# df_sow202_updated = df_sow202_updated_CpG_98.copy()


df_sow202_updated.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,CpG_lung_score,pred_frac_force
26,SOW202,130920290_0,A1187780,B00817816,130920290,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145737441.0,19,F1082F,G>A,c.3246C>T,NM_004260.4,51.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748604051,NaN,NaN,NaN,NaN,4120.0,1.11,NaN,NaN,Detected,0.010985,3.86,Low,Not Detected,98.70,2.5,0.0,10/23/2024,07/05/2023,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A00770_1052_BHTCLWDSXC,/ghsfa/ivd/flowcentral/241101_A00770_1052_BHTCLWDSXC.79230695-5143-4a73-b52e-a27b1400da85.20241103143625/sirius,-20.639350,0.010985
65,SOW202,34095_0,A1187819,B00817817,34095,0,1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,SNV,NaN,TFE3,X,48887805.0,10,G531E,C>T,c.1592G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs782285732,NaN,NaN,NaN,NaN,263.0,48.40,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,2.56,3.0,0.0,10/23/2024,02/28/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-21.641157,0.000143
70,SOW202,34800_0,A1187824,B00817818,34800,0,1,SUCCESS,NaN,SNV,NaN,CIC,19,42777840.0,2,P635P,A>G,c.1905A>G,NM_001304815.1,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs926431641,NaN,NaN,NaN,NaN,3204.0,5.59,NaN,NaN,Not Detected,0.0,10.40,Low,Not Detected,24.47,3.0,0.0,10/23/2024,05/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-22.053279,0.00012
71,SOW202,34070_0,A1187825,B00817819,34070,0,1,SUCCESS,NaN,SNV,NaN,CDKN1C,11,2906208.0,1,A171V,G>A,c.512C>T,NM_000076.2,80.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,47.17,NaN,NaN,Detected,0.041027,6.62,Low,Not Detected,61.32,3.0,0.0,10/23/2024,02/29/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.212699,0.041027
68,SOW202,34118_0,A1187822,B00817820,34118,0,1,SUCCESS,NaN,SNV,NaN,ARID1B,6,157222571.0,3,Y600C,A>G,c.1799A>G,NM_017519.3,50.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4508.0,0.24,NaN,NaN,Detected,0.000451,NaN,Not evaluable,Not Detected,91.10,2.5,0.0,10/23/2024,04/05/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable,241101_A01020_0917_AHTC52DSXC,/ghsfa/ivd/flowcentral/241101_A01020_0917_AHTC52DSXC/sirius,-20.461339,0.000451


In [29]:
# Genearte the final report

# Subset the updated report for updated MB score and TFv2gamma columns
df_sow202_5_sub = df_sow202_updated[['GHSampleID', 'Tumor_methylation_status', 'Tumor_methylation_score']]
df_sow202_5_sub_dedup = df_sow202_5_sub.drop_duplicates(subset = 'GHSampleID')
print("Total sample size of SOW202 Infinity cumulative batches with updated MB score and TFv2gamma is " + str(df_sow202_5_sub_dedup.shape[0]) + "\n") 

# Drop duplicates of original report
df_sow202_all_dedup = df_sow202_all.drop_duplicates(subset = "GHSampleID")
print("Total sample size of original SOW187 Infinity cumulative batches is " + str(df_sow202_all_dedup.shape[0]) + "\n") 

# Check failed samples
df_sow202_all_dedup.Sample_status.value_counts()


Total sample size of SOW202 Infinity cumulative batches with updated MB score and TFv2gamma is 77

Total sample size of original SOW187 Infinity cumulative batches is 80



SUCCESS    77
FAIL        3
Name: Sample_status, dtype: int64

In [30]:
# For Infinity report

# Merge the updated values into df_sow??_final based on 'GHSampleID'
df_merged = df_sow202_all.merge(df_sow202_5_sub_dedup[['GHSampleID', 'Tumor_methylation_status', 'Tumor_methylation_score']],
                                  on='GHSampleID', 
                                  how='left', 
                                  suffixes=('', '_updated'))

# Update the original columns with the values from the merged columns if they are not NaN
df_merged['Tumor_methylation_status'] = df_merged['Tumor_methylation_status_updated'].combine_first(df_merged['Tumor_methylation_status'])
df_merged['Tumor_methylation_score'] = df_merged['Tumor_methylation_score_updated'].combine_first(df_merged['Tumor_methylation_score'])


# Drop the temporary updated columns
df_merged.drop(columns=['Tumor_methylation_status_updated', 'Tumor_methylation_score_updated'], inplace=True)

# Ensure the dataframe maintains the original order and columns of df_sow??_final
df_sow202_infinity_final_all = df_merged[df_sow202_all.columns]

# Save the data
df_sow202_infinity_final_all.to_csv(f"{work_dir}/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG97.csv", index = False)

df_sow202_infinity_final_all.head()


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,TNFAIP3,6,138202447.0,9,M788I,G>A,c.2364G>A,NM_001270507.2,52.65,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52800632,rs143002189,NaN,NaN,NaN,NaN,1176.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
1,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,DOT1L,19,2226694.0,27,G1392S,G>A,c.4174G>A,NM_032482.3,51.59,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs375002753,NaN,NaN,NaN,NaN,1131.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
2,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,RET,10,43615594.0,15,S891S,G>A,c.2673G>A,NM_020975.6,51.21,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs201620214,Likely_benign,36302.0,NaN,NaN,1428.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
3,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,ARID2,12,46233253.0,11,T491S,C>G,c.1472C>G,NM_152641.4,50.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV104636884,rs149755754,Likely_benign,137297.0,NaN,NaN,1313.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable
4,SOW202,34101_0,A1187754,B00817859,34101,0,0,SUCCESS,NaN,SNV,NaN,CREBBP,16,3779687.0,31,N1787N,G>A,c.5361C>T,NM_004380.3,49.92,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52129505,rs375462934,Uncertain_significance,100953.0,NaN,NaN,1328.0,0.0,NaN,NaN,Not Detected,0.0,NaN,Not evaluable,Not Detected,7.83,3.0,0.0,10/23/2024,03/01/2024,11/14/2024,Non-small cell lung carcinoma (NSCLC),Not Applicable,Not Applicable


In [31]:
def suppress_methylation_score_higher_than_1(df, score_column='Tumor_methylation_score', id_column='GHSampleID'):
    # Identify rows where the score is greater than 1
    rows_to_modify = df[df[score_column] > 1].copy()

    # Remove duplicates based on GHSampleID and Tumor_methylation_score
    unique_rows_to_modify = rows_to_modify[[id_column, score_column]].drop_duplicates()

    # Print the unique rows that will be modified with their original scores and sample IDs
    if not unique_rows_to_modify.empty:
        print("Unique rows with scores > 1 before modification:")
        print(unique_rows_to_modify)

    # Modify the dataframe by capping the scores at 0.99
    df[score_column] = np.where(df[score_column] > 1, 0.99, df[score_column])

    # Show the rows after modification, ensuring they are also unique
    modified_rows = df.loc[rows_to_modify.index, [id_column, score_column]].drop_duplicates()
    if not modified_rows.empty:
        print("\nUnique rows after modification:")
        print(modified_rows)

    return df

In [32]:
# Add TFv2 suppression rule. Apply secondary rule to corrct TFv2 score > 1 to 0.99
# df_sow202_infinity_final_all = pd.read_csv(f"{work_dir}/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport.csv")

# Ad-hoc process
df_sow202_infinity_final_all_new = suppress_methylation_score_higher_than_1(df_sow202_infinity_final_all)

# Save the data
df_sow202_infinity_final_all_new.to_csv(f"{work_dir}/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG97.csv", index = False)

In [33]:
# Check the detection

# Define file paths
file_paths = [
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB90.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB95_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB95.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB97_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB97.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB98.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG90.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG95.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG97.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW202_11252024/20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG98.csv",

]

# Initialize an empty list to store the counts
summary = []

# Loop through each file
for file_path in file_paths:
    # Read the CSV file
    df = pd.read_csv(file_path)
    # Remove duplicates
    df = df.drop_duplicates(subset = "GHSampleID")

    # Count the values in the "Tumor_methylation_status" column
    status_counts = df["Tumor_methylation_status"].value_counts()
    detected = status_counts.get("Detected", 0)
    not_detected = status_counts.get("Not Detected", 0)
    total = detected + not_detected
    
    # Calculate detection rate
    detection_rate = (detected / total) * 100 if total > 0 else 0
    
    # Extract specificity from the file name
    specificity = "_".join(file_path.split("/")[-1].split("_")[-2:])
    
    # Add the counts and new columns to the summary
    summary.append({
        "File": file_path.split("/")[-1],
        "Detected": detected,
        "Not Detected": not_detected,
        "Detection Rate %": round(detection_rate, 2),
        "Specificity": specificity
    })

# Convert the summary into a DataFrame for better visualization
summary_df = pd.DataFrame(summary)

# Display the result
summary_df


,File,Detected,Not Detected,Detection Rate %,Specificity
0,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB90.csv,35,42,45.45,InfinityReport_MB90.csv
1,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB95_withTFv2filter.csv,35,42,45.45,MB95_withTFv2filter.csv
2,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB95.csv,30,47,38.96,InfinityReport_MB95.csv
3,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB97_withTFv2filter.csv,34,43,44.16,MB97_withTFv2filter.csv
4,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB97.csv,28,49,36.36,InfinityReport_MB97.csv
5,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_MB98.csv,27,50,35.06,InfinityReport_MB98.csv
6,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG90.csv,38,39,49.35,InfinityReport_CpG90.csv
7,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG95.csv,35,42,45.45,InfinityReport_CpG95.csv
8,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG97.csv,33,44,42.86,InfinityReport_CpG97.csv
9,20241125_AstraZeneca_SOW202_80Samples_TFv2_InfinityReport_CpG98.csv,33,44,42.86,InfinityReport_CpG98.csv
